In [ ]:
#installing pyspark 
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 51.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=c43fa126bccc5d7a9e1a3e5aea644623185ee69fa3e5d1297a727e5e73a268b7
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
#importing required libraries
from pyspark import SparkContext,SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions as f
from pyspark.sql.types import IntegerType

In [ ]:
#Spark Config
conf = SparkConf().setAppName('airline_conf')
sc = SparkContext(conf=conf)
spark=SparkSession.builder.appName('spark_airline').getOrCreate()
sqlcontext=SQLContext(spark)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [ ]:
#loading data set
df=spark.read.csv('/content/Airline_data.csv',header=True)

In [ ]:
#Q1. Showing sample 5 records from dataset
df.show(5)

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|1989|    1|        23|        1|   1419|      1230|   1742|      1552|           UA|      183

In [ ]:
#Q2. Read the data with data types
df.printSchema()

root
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: string (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: string (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- CarrierDelay:

In [ ]:
#Q3 Make a new column MonthStr, Which has months in form of 01, 02, 03, ..., 12.
modified_df=df.withColumn("MonthStr",f.date_format(f.to_date(f.concat_ws('-',df.Year,df.Month,df.DayofMonth)),"MM"))
modified_df.select('MonthStr').show()

+--------+
|MonthStr|
+--------+
|      01|
|      01|
|      01|
|      01|
|      01|
|      01|
|      01|
|      01|
|      01|
|      01|
|      01|
|      01|
|      01|
|      01|
|      01|
|      01|
|      01|
|      01|
|      01|
|      01|
+--------+
only showing top 20 rows



In [ ]:
#Q4 Find the # of flights each airline made.
df.groupBy('UniqueCarrier').count().orderBy(f.desc('count')).show()

+-------------+-----+
|UniqueCarrier|count|
+-------------+-----+
|           UA|  426|
+-------------+-----+



In [ ]:
# changing data type of column DepDelay and ArrDelay, converting this column into integer
modified_df=df.withColumn("DepDelay", df["DepDelay"].cast(IntegerType())) 
modified_df=modified_df.withColumn("ArrDelay", modified_df["ArrDelay"].cast(IntegerType())) 

In [ ]:
modified_df.printSchema()
#checking datatype after converting it

root
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: string (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: string (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: integer (nullable = true)
 |-- DepDelay: integer (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- CarrierDela

In [ ]:
#Q5 Find the mean arrival delay per origination airport?
modified_df.groupBy('Origin').mean('ArrDelay').withColumnRenamed('avg(ArrDelay)','avg_ArrDelay').show()

+------+-------------------+
|Origin|       avg_ArrDelay|
+------+-------------------+
|   LIH|0.16666666666666666|
|   HNL|  14.21774193548387|
|   EWR|               9.25|
|   DEN| 20.166666666666668|
|   IAD| 12.966666666666667|
|   SFO| 11.215384615384615|
|   PHL|  6.827586206896552|
|   OGG|  16.24137931034483|
+------+-------------------+



In [ ]:
#Q6 What is the average departure delay from each airport?
modified_df.groupBy('Origin').avg('DepDelay').withColumnRenamed('avg(DepDelay)','avg_DepDelay').show()

+------+-------------------+
|Origin|       avg_DepDelay|
+------+-------------------+
|   LIH|-3.7666666666666666|
|   HNL|  3.217741935483871|
|   EWR|  4.958333333333333|
|   DEN|               27.6|
|   IAD|                8.9|
|   SFO| 19.646153846153847|
|   PHL| 16.137931034482758|
|   OGG|                6.0|
+------+-------------------+



In [ ]:



#Q3 Make a new column MonthStr, Which has months in form of 01, 02, 03, ..., 12.
modified_df=df.withColumn("MonthStr",f.date_format(f.to_date(f.concat_ws('-',df.Year,df.Month,df.DayofMonth)),"MM"))
modified_df.select('MonthStr').show()

In [ ]:
df1=df.withColumn("MonthStr",f.concat_ws('-',df.Year,df.Month,df.DayofMonth))
df2=df1.withColumn("MonthStr1",f.to_date(df1.MonthStr))
df3=df2.withColumn("MonthStr3",f.date_format( df2.MonthStr1,"MM"))
df3.select('MonthStr3').show(2)

+---------+
|MonthStr3|
+---------+
|       01|
|       01|
+---------+
only showing top 2 rows

